## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Dropping more or fewer columns
app_df = application_df.drop(columns=["EIN", "STATUS","SPECIAL_CONSIDERATIONS"])
app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
app_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [4]:
# Creating more bins for rare occurrences in columns.
name_count = app_df['NAME'].value_counts()
name_count

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [5]:
# Choose a cutoff value
names_to_replace = list(name_count[name_count < 100].index)

# Replace in dataframe
for app in names_to_replace:
    app_df['NAME'] = app_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
app_df['NAME'].value_counts()

Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(app_df)
dummies_df

,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,NAME_HONOR SOCIETY OF PHI KAPPA PHI,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(columns='IS_SUCCESSFUL')
y = dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
length = X_train.shape[1]
length

144

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, input_dim=length))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 80)                11600     
                                                                 
 dense_6 (Dense)             (None, 30)                2430      
                                                                 
 dense_7 (Dense)             (None, 15)                465       
                                                                 
 dense_8 (Dense)             (None, 15)                240       
                                                                 
 dense_9 (Dense)             (None, 1)                 16        
                                                                 
Total params: 14751 (57.62 KB)
Trainable params: 14751 (57.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 825us/step - loss: 0.4720 - accuracy: 0.7666
Epoch 2/100
804/804 [==============================] - 1s 820us/step - loss: 0.4720 - accuracy: 0.7683
Epoch 3/100
804/804 [==============================] - 1s 774us/step - loss: 0.4722 - accuracy: 0.7673
Epoch 4/100
804/804 [==============================] - 1s 808us/step - loss: 0.4724 - accuracy: 0.7668
Epoch 5/100
804/804 [==============================] - 1s 785us/step - loss: 0.4715 - accuracy: 0.7684
Epoch 6/100
804/804 [==============================] - 1s 796us/step - loss: 0.4713 - accuracy: 0.7675
Epoch 7/100
804/804 [==============================] - 1s 807us/step - loss: 0.4711 - accuracy: 0.7692
Epoch 8/100
804/804 [==============================] - 1s 792us/step - loss: 0.4714 - accuracy: 0.7678
Epoch 9/100
804/804 [==============================] - 1s 775us/step - loss: 0.4717 - accuracy: 0.7664
Epoch 10/100
804/804 [==============================] - 1s 787us/step - l

804/804 [==============================] - 1s 798us/step - loss: 0.4694 - accuracy: 0.7696
Epoch 80/100
804/804 [==============================] - 1s 792us/step - loss: 0.4683 - accuracy: 0.7699
Epoch 81/100
804/804 [==============================] - 1s 802us/step - loss: 0.4686 - accuracy: 0.7697
Epoch 82/100
804/804 [==============================] - 1s 791us/step - loss: 0.4696 - accuracy: 0.7705
Epoch 83/100
804/804 [==============================] - 1s 801us/step - loss: 0.4713 - accuracy: 0.7694
Epoch 84/100
804/804 [==============================] - 1s 804us/step - loss: 0.4691 - accuracy: 0.7702
Epoch 85/100
804/804 [==============================] - 1s 809us/step - loss: 0.4687 - accuracy: 0.7704
Epoch 86/100
804/804 [==============================] - 1s 796us/step - loss: 0.4687 - accuracy: 0.7693
Epoch 87/100
804/804 [==============================] - 1s 806us/step - loss: 0.4688 - accuracy: 0.7693
Epoch 88/100
804/804 [==============================] - 1s 779us/step - loss:

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4993 - accuracy: 0.7586 - 234ms/epoch - 873us/step
Loss: 0.49933406710624695, Accuracy: 0.7586005926132202


In [20]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")